In [ ]:
import polars as pl
from datetime import datetime,timedelta
import get_and_store as gas
import polars_indicators as pi
from polars_indicators.indicators import IndicatorResult
from importlib import reload
DATE_COLUNN = "Date"

In [ ]:
reload(pi)
reload(pi.strategies)
reload(pi.strategies.buy_x_week_low)
reload(pi.strategies.buy_close_trail)
reload(pi.summarize.trades)
reload(pi.indicators)

In [ ]:
try:
    df = saved_df.clone()
except NameError:
        df = gas.get_all_saved_data_df()
        df = df.filter(pl.col(pi.indicators.DATE_COLUMN) >= datetime(2020, 1, 1))
        df = df.collect()

In [ ]:
saved_df = df.clone()

In [ ]:
df = df.with_columns(pl.col(pi.indicators.DATE_COLUMN).cast(pl.Datetime))

In [ ]:
lower_limit = 3
upper_limit = 250
close_within_limits = f"close_over{lower_limit}_under{upper_limit}"
df = df.with_columns(((pl.col(pi.indicators.CLOSE_COLUMN) > lower_limit) & (pl.col(pi.indicators.CLOSE_COLUMN) < upper_limit)).alias(close_within_limits))

In [ ]:
sma_pairs = [(3, 5), (5, 8), (8, 21), (10, 20), (15, 30), (50, 100)]

In [ ]:
column = pi.indicators.CLOSE_COLUMN

In [ ]:
indicator_pairs = []
for sma_pair in sma_pairs:
    sma0 = pi.indicators.simple_moving_average(df, sma_pair[0], column)
    sma1 = pi.indicators.simple_moving_average(sma0.df, sma_pair[1], column)
    indicator_pairs.append((sma0, sma1))
    df = sma1.df

In [ ]:
buy_indicators = []
for indicator_pair in indicator_pairs:
    ind = pi.indicators.crossover_up(df, indicator_pair[0].column, indicator_pair[1].column)
    buy_indicators.append(ind)
    df = ind.df.with_columns((pl.col(ind.column) & pl.col(close_within_limits)).alias(ind.column))

In [ ]:
russell = df.filter(pl.col(pi.indicators.SYMBOL_COLUMN).is_in(pi.Russell1000_2020))

In [ ]:
df = df.filter(pl.col(pi.indicators.SYMBOL_COLUMN).is_in(pi.SnP500_2020))

In [ ]:
strat_results = [pi.summarize.trades.Trades.from_strategy_result(pi.strategies.buy_close_trail.strategy(df, buy_indicator.column, trailing_stop_bars=2)) for buy_indicator in buy_indicators]

In [ ]:
pi.summarize.trades.compare_strategies(strat_results, [str(x) for x in sma_pairs])

In [ ]:
russell_results = [pi.summarize.trades.Trades.from_strategy_result(pi.strategies.buy_close_trail.strategy(russell, buy_indicator.column, trailing_stop_bars=2)) for buy_indicator in buy_indicators]

In [ ]:
pi.summarize.trades.compare_strategies(russell_results, [str(x) for x in sma_pairs])